# Preparing the data

## Importing libraries, downloading the model

In [ ]:
import pandas
import sklearn
import numpy

print(pandas.__version__)
print(sklearn.__version__)
print(numpy.__version__)

1.5.3
1.2.2
1.22.4


In [ ]:
import spacy
# Small Russian model:
# !python -m spacy download ru_core_news_sm
# nlp = spacy.load('ru_core_news_sm')

# Large Russian model:
!python -m spacy download ru_core_news_lg
nlp = spacy.load('ru_core_news_lg')

2023-06-19 18:06:02.843617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=985f839c47376b42d048bfa1e463ab58197d4cd9e6d108e2e06818ee42df3407
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


## Making lists and doc objects from csv files

In [ ]:
# Let's load the training data from a csv file
train_set = pandas.read_csv('./train_data.csv', encoding='utf-8')
# train_set

In [ ]:
test_set = pandas.read_csv('./test_data.csv', encoding='utf-8')
# test_set

In [ ]:
train_sentences = train_set['text'].to_list()
train_authors = train_set['author'].to_list()

test_sentences = test_set['text'].to_list()
test_authors = test_set['author'].to_list()

print(len(train_authors), len(test_authors))

10000 1000


## Normalizing the text, replacing "..." with "…"

In [ ]:
def replace_ellipsis(sentences):
  updated_sentences = []
  for sentence in sentences:
    updated_sentence = sentence.replace("...", "…")
    updated_sentences.append(updated_sentence)
  return updated_sentences

In [ ]:
# Extract data for Dostoyevsky
dostoyevsky_data = train_set[train_set['author'] == 'Dostoevsky']['text'].to_list()

# Extract data for Tolstoy
tolstoy_data = train_set[train_set['author'] == 'Tolstoy']['text'].to_list()

# Extract data for Chekhov
chekhov_data = train_set[train_set['author'] == 'Chekhov']['text'].to_list()

# Extract data for Gogol
gogol_data = train_set[train_set['author'] == 'Gogol']['text'].to_list()

train_sentences = replace_ellipsis(train_sentences)
test_sentences = replace_ellipsis(test_sentences)

dostoyevsky_data = replace_ellipsis(dostoyevsky_data)
tolstoy_data = replace_ellipsis(tolstoy_data)
chekhov_data = replace_ellipsis(chekhov_data)
gogol_data = replace_ellipsis(gogol_data)

In [ ]:
train_doc_sentences = nlp.pipe(train_sentences)
test_doc_sentences = nlp.pipe(test_sentences)

dostoyevsky_data_doc = nlp.pipe(dostoyevsky_data)
tolstoy_data_doc = nlp.pipe(tolstoy_data)
chekhov_data_doc = nlp.pipe(chekhov_data)
gogol_data_doc = nlp.pipe(gogol_data)

## Preparing the feature set

In [ ]:
from collections import Counter

def fivegram_pos_count(list_of_doc_sentences):
  n = 5
  fivegram_pos_tags = []
  for doc in list_of_doc_sentences:
    # Iterate over each possible fivegram in the document
    for i in range(len(doc) - n + 1):
            # Extract the tokens for the current fivegram
            fivegram_tokens = doc[i : i + n]
            # Extract the POS tags of the tokens and add the POS tag combination to the list
            fivegram_pos = tuple(token.pos_ for token in fivegram_tokens)
            fivegram_pos_tags.append(fivegram_pos)
  most_common_fivegrams = Counter(fivegram_pos_tags).most_common(5)
  five_fivegrams_list = [element[0] for element in most_common_fivegrams]
  return five_fivegrams_list

def fivegram_pos_extractor_from_sentence(doc):
    n = 5
    fivegram_pos_tags = []
    for i in range(len(doc) - n + 1):
    # Extract the tokens for the current fivegram
        fivegram_tokens = doc[i : i + n]
        # Extract the POS tags of the tokens and add the POS tag combination to the list
        fivegram_pos = tuple(token.pos_ for token in fivegram_tokens)
        fivegram_pos_tags.append(fivegram_pos)
    unique_fivegram_pos_tags = list(set(fivegram_pos_tags))

    return unique_fivegram_pos_tags

In [ ]:
from collections import Counter
five_fivegrams_dostoyevsky = fivegram_pos_count(dostoyevsky_data_doc)
five_fivegrams_tolstoy = fivegram_pos_count(tolstoy_data_doc)
five_fivegrams_chekhov = fivegram_pos_count(chekhov_data_doc)
five_fivegrams_gogol = fivegram_pos_count(gogol_data_doc)

fivegrams_list = five_fivegrams_dostoyevsky + five_fivegrams_tolstoy + five_fivegrams_chekhov + five_fivegrams_gogol
fivegrams_list = list(set(fivegrams_list))

In [ ]:
print(five_fivegrams_dostoyevsky)
print(five_fivegrams_tolstoy)
print(five_fivegrams_chekhov)
print(five_fivegrams_gogol)
print(len(fivegrams_list), fivegrams_list)

[('VERB', 'ADP', 'DET', 'NOUN', 'PUNCT'), ('PUNCT', 'SPACE', 'PUNCT', 'VERB', 'PRON'), ('NOUN', 'PUNCT', 'SPACE', 'PUNCT', 'SPACE'), ('VERB', 'PUNCT', 'SPACE', 'PUNCT', 'SPACE'), ('VERB', 'PRON', 'ADP', 'NOUN', 'PUNCT')]
[('X', 'X', 'X', 'X', 'X'), ('PUNCT', 'VERB', 'ADP', 'NOUN', 'PUNCT'), ('X', 'X', 'X', 'X', 'PUNCT'), ('NOUN', 'PUNCT', 'VERB', 'ADP', 'NOUN'), ('NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT')]
[('NOUN', 'PUNCT', 'VERB', 'ADP', 'NOUN'), ('PUNCT', 'VERB', 'ADP', 'NOUN', 'PUNCT'), ('PUNCT', 'PUNCT', 'VERB', 'PRON', 'PUNCT'), ('NOUN', 'VERB', 'ADP', 'NOUN', 'PUNCT'), ('ADJ', 'PUNCT', 'ADJ', 'NOUN', 'PUNCT')]
[('NOUN', 'PUNCT', 'VERB', 'ADP', 'NOUN'), ('PUNCT', 'VERB', 'ADP', 'NOUN', 'PUNCT'), ('VERB', 'ADP', 'ADJ', 'NOUN', 'PUNCT'), ('NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT'), ('NOUN', 'VERB', 'ADP', 'NOUN', 'PUNCT')]
14 [('VERB', 'PUNCT', 'SPACE', 'PUNCT', 'SPACE'), ('VERB', 'PRON', 'ADP', 'NOUN', 'PUNCT'), ('X', 'X', 'X', 'X', 'X'), ('NOUN', 'PUNCT', 'VERB', 'ADP', 'NOUN'), ('ADJ', 

In [ ]:
import string

new_punctuation_string = "—«»–‹›…" + string.punctuation
print(new_punctuation_string)
print(len(new_punctuation_string))

french_accent_marks = "éèêëàÉÈÊËÀ"
new_string = string.ascii_letters + french_accent_marks
print(len(new_string), new_string)

final_string = new_punctuation_string + new_string
print(len(final_string), final_string)

—«»–‹›…!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
39
62 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéèêëàÉÈÊËÀ
101 —«»–‹›…!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéèêëàÉÈÊËÀ


In [ ]:
import numpy

entity_types = ['PER', 'LOC', 'ORG']

feature_nr = 3 + len(fivegrams_list) + len(final_string)
print(feature_nr)

# We are creating a matrix with zero vectors for each review (in training set and test set)
train_features_matrix = numpy.zeros((len(train_sentences), feature_nr))
print(train_features_matrix.shape)

test_features_matrix = numpy.zeros((len(test_sentences), feature_nr))
print(test_features_matrix.shape)

118
(10000, 118)
(1000, 118)


# Modifying the feature vectors

## Visualisation tests

In [ ]:
import sys

train_doc_sentences = nlp.pipe(train_sentences)

counter = 0

for sentence, author in zip(train_doc_sentences, train_authors):
    print(author)
    print(sentence)
    NEs_in_sentence = [entity.label_ for entity in sentence.ents]
    print(NEs_in_sentence)

#NER:
    for entity_type in entity_types:
      if entity_type in NEs_in_sentence:
        print(entity_type)
        entity_id = entity_types.index(entity_type)
        print(entity_id)
    fivegram_pos_list = fivegram_pos_extractor_from_sentence(sentence)
    print(fivegram_pos_list)

#Common 5-gram POS:
    for fivegram in fivegrams_list:
      if fivegram in fivegram_pos_list:
        print(fivegram)
        fivegram_id = fivegrams_list.index(fivegram) + 3
        print(fivegram_id)

#Punctuation + Latin characters:
      for char in sentence.text:
        if char in final_string:
          print(char)
          char_id = final_string.index(char) + 3 + len(fivegrams_list)
          print(char_id)

    print()
    counter +=1
    if counter == 2:
        break

Visualisation test 2:

In [ ]:
# Process a text
train_features_matrix = numpy.zeros((len(train_sentences), feature_nr))
train_doc_sentences = nlp.pipe(train_sentences)

counter = 0
# loop over each review, label and feature vector at the same time (zip)
for sentence, author, feature_vector in zip(train_doc_sentences, train_authors, train_features_matrix):
    print('Author:', author)
    print(sentence)
    NEs_in_sentence = [entity.label_ for entity in sentence.ents]

#NER:
    for entity_type in entity_types:
      if entity_type in NEs_in_sentence:
        entity_id = entity_types.index(entity_type)
        print(entity_type)
        print(entity_id)
        feature_vector[entity_id] = 1
        print(feature_vector)
    fivegram_pos_list = fivegram_pos_extractor_from_sentence(sentence)
    print(fivegram_pos_list)

#Common 5-gram POS:
    for fivegram in fivegrams_list:
      if fivegram in fivegram_pos_list:
        fivegram_id = fivegrams_list.index(fivegram) + 3
        print(fivegram_id)
        print(fivegram)
        feature_vector[fivegram_id] = 1
        print(feature_vector)

#Punctuation + Latin characters:
    for char in sentence.text:
      if char in final_string:
        print(char)
        char_id = final_string.index(char) + 3 + len(fivegrams_list)
        feature_vector[char_id] = 1
        print(char_id)
        print(feature_vector)

    print()
    counter +=1
    if counter == 2:
        break

Author: Dostoevsky
Но каково же было мое изумление, когда Наташа с первых же слов остановила меня и сказала, что нечего ее утешать, что она уже пять дней, как знает про это..     – Боже мой!
PER
0
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[('ADV', 'NUM', 'NOUN', 'PUNCT', 'SCONJ'), ('PUNCT', 'SCONJ', 'PROPN', 'ADP', 'ADJ'), ('ADP', 'ADJ', 'PART', 'NOUN', 'VERB'), ('SPACE', 'PUNCT', 'NOUN', 'DET', 'PUNCT'), ('NUM', 'NOUN', 'PUNCT', 'SCONJ', 'VERB'), ('ADJ', 'PART', 'AUX', 'DET', 'NOUN'), ('PRON', 'PUNCT', 'SPACE', 'PUNCT', 'NOUN'), ('NOUN', 'PUNCT', 'SCONJ', 'VERB', 'ADP'), ('PUNCT', 'SCONJ', 'VERB', 'ADP', 'PRON'), ('SCONJ', 'PROPN', 'ADP', 'ADJ', 'PART'), ('PUNCT', 'SCONJ',

# Writing a function for feature vector modification

In [ ]:
def modify_feature_vectors(doc_sentences, features_matrix):
  for sentence, feature_vector in zip(doc_sentences, features_matrix):
    NEs_in_sentence = [entity.label_ for entity in sentence.ents]

#NER:
    for entity_type in entity_types:
      if entity_type in NEs_in_sentence:
        entity_id = entity_types.index(entity_type)
        feature_vector[entity_id] = 1

#Common 5-gram POS:
    fivegram_pos_list = fivegram_pos_extractor_from_sentence(sentence)
    for fivegram in fivegrams_list:
      if fivegram in fivegram_pos_list:
        fivegram_id = fivegrams_list.index(fivegram) + 3
        feature_vector[fivegram_id] = 1

#Punctuation + Latin characters:
    for char in sentence.text:
      if char in final_string:
        char_id = final_string.index(char) + 3 + len(fivegrams_list)
        feature_vector[char_id] = 1

  return features_matrix

In [ ]:
train_features_matrix = numpy.zeros((len(train_sentences), feature_nr))
train_doc_sentences = nlp.pipe(train_sentences)

In [ ]:
train_features_matrix_final = modify_feature_vectors(train_doc_sentences, train_features_matrix)

# Training

In [ ]:
from sklearn.linear_model import LogisticRegression

lr8_all_features = LogisticRegression()

# Train the model on the data, storing the information learned from the dat`a
# Model is learning the relationship between digits (x_train) and labels (y_train)
lr8_all_features.fit(train_features_matrix_final, train_authors)

print(lr8_all_features.classes_)
print(lr8_all_features.get_params())

['Chekhov' 'Dostoevsky' 'Gogol' 'Tolstoy']
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Modifying the test set feature vectors

In [ ]:
test_doc_sentences = nlp.pipe(test_sentences)
test_features_matrix = numpy.zeros((len(test_sentences), feature_nr))

test_features_matrix_final = modify_feature_vectors(test_doc_sentences, test_features_matrix)

# Making predictions

In [ ]:
def predict(i):
    print(test_sentences[i])
    # print the features of the index
    print(test_features_matrix_final[i])
    # print the correct label of the index
    print(test_authors[i])

    print()
    print("Prediction:")
    # print the prediction for the features of this index
    print(lr8_all_features.predict([test_features_matrix_final[i]]))
    # print the probabilities for each label predictions
    print(lr8_all_features.predict_proba([test_features_matrix_final[i]]))
    print()

In [ ]:
predict(0)
predict(1)
predict(2)
predict(3)
predict(4)
predict(5)

"Фома Фомич, говорю, разве это возможное дело?
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Dostoevsky

Prediction:
['Tolstoy']
[[0.01471629 0.3816194  0.12940678 0.47425753]]

Пора бы уже домой.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Chekhov

Prediction:
['Tolstoy']
[[0.23393698 0.23732886 0.25183665 0.27689751]]

А казаки все до одного прощались, зная, что мно

In [ ]:
test_predictions_all_features = lr8_all_features.predict(test_features_matrix_final)

for p, r in zip(test_predictions_all_features[:10], test_authors[:10]):
    if p == r:
        result = "Correct"
    else:
        result = "Incorrect"
    print(p + "(" + result + ":" + r + ")")

Tolstoy(Incorrect:Dostoevsky)
Tolstoy(Incorrect:Chekhov)
Gogol(Correct:Gogol)
Dostoevsky(Correct:Dostoevsky)
Dostoevsky(Incorrect:Gogol)
Tolstoy(Incorrect:Dostoevsky)
Dostoevsky(Correct:Dostoevsky)
Chekhov(Correct:Chekhov)
Dostoevsky(Correct:Dostoevsky)
Gogol(Incorrect:Dostoevsky)


# Saving the model

In [ ]:
import pickle

# Save to file in the current working directory
pkl_filename = "logreg8_all_features.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr8_all_features, file)

# Evaluating the model

## Dummy model

In [ ]:
from sklearn.metrics import accuracy_score

dummy_predictions = ['Dostoevsky'] * len(test_sentences)
print(len(dummy_predictions))

# Calculate the accuracy of these "dummy predictions"
acc_dummy = accuracy_score(test_authors, dummy_predictions)
print('The accuracy is:', acc_dummy)
print()

from sklearn.metrics import classification_report
print(classification_report(test_authors, dummy_predictions))

1000
The accuracy is: 0.25

              precision    recall  f1-score   support

     Chekhov       0.00      0.00      0.00       250
  Dostoevsky       0.25      1.00      0.40       250
       Gogol       0.00      0.00      0.00       250
     Tolstoy       0.00      0.00      0.00       250

    accuracy                           0.25      1000
   macro avg       0.06      0.25      0.10      1000
weighted avg       0.06      0.25      0.10      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NER + Common POS Model

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy:')

acc = accuracy_score(test_authors, test_predictions_all_features)
print(acc)
corr_count = accuracy_score(test_authors, test_predictions_all_features, normalize=False)
total_count = len(test_authors)

print("Total reviews: " + str(str(total_count)))
print("Total correct predictions:" + str(corr_count))
corr_ratio = corr_count / total_count
print("Correct ratio:" + str(corr_ratio))

Accuracy:
0.394
Total reviews: 1000
Total correct predictions:394
Correct ratio:0.394


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_authors, test_predictions_all_features))

              precision    recall  f1-score   support

     Chekhov       0.55      0.43      0.48       250
  Dostoevsky       0.32      0.38      0.35       250
       Gogol       0.42      0.34      0.38       250
     Tolstoy       0.34      0.43      0.38       250

    accuracy                           0.39      1000
   macro avg       0.41      0.39      0.40      1000
weighted avg       0.41      0.39      0.40      1000

